Análise inicial

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('../data/raw/iFood.xlsx')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.value_counts()

In [ ]:
df.describe(include='object')

---

Preprocessamento e Limpeza

In [ ]:
df.columns

In [ ]:
df['MntRegularProds'].head()

In [ ]:
# removendo os produtos regulares comprados e os registros da aceitação do cliente

df.drop(columns=['MntRegularProds','AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3',
       'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmpOverall'], inplace=True)

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.rename(columns={
    'Index': 'ID',
    'Income': 'Salario',
    'Age': 'Idade',
    'Customer_Days': 'Dias-Cliente',
    'marital_status': 'Estado_Civil',
    'education': 'Formação',
    'Kidhome': 'Criancas-Casa',
    'Teenhome': 'Adolescentes-Casa',
    'Recency': 'Recencia',
    'MntWines': 'Qtde-Vinhos',
    'MntFruits': 'Qtde-Frutas',
    'MntMeatProducts': 'Qtde-Carnes',
    'MntFishProducts': 'Qtde-Peixes',
    'MntSweetProducts': 'Qtde-Doces',
    'MntGoldProds': 'Qtde-Premium',
    'MntTotal': 'Gasto-Cliente',
    'NumDealsPurchases': 'Qtde-Compras',
    'NumWebPurchases': 'Qtde-Compras-Web',
    'NumCatalogPurchases': 'Qtde-Compras-Catalogo',
    'NumStorePurchases': 'Qtde-Compras-Loja',
    'NumWebVisitsMonth': 'Visitas-Site-Mes',
    'Complain': 'Reclamacoes',
}, inplace=True)

In [ ]:
df['ID'] = df['ID'] - 1000

df['ID'].unique()

In [ ]:
df.columns

In [ ]:
df['Formação'].unique()

In [ ]:
df['Formação'] = df['Formação'].replace({
    'Graduation': 'Graduação',
    'PHD': 'Doutorado',
    'Master': 'Mestrado',
    'Basic': 'Fundamental',
    '2n Cycle': 'Medio'
})

In [ ]:
df['Estado_Civil'].unique()

In [ ]:
df['Estado_Civil'] = df['Estado_Civil'].replace({
    'Single': 'Solteiro',
    'Together': 'Namorando',
    'Married': 'Casado',
    'Divorced': 'Divorciado',
    'Widow': 'Viúvo(a)'
})

In [ ]:
caixas = [20, 30, 40, 50, 60, 70, 80]

rotulos = [
    '21-30', '31-40','41-50','51-60','61-70','71-80'
]

df['Faixa-Etaria'] = pd.cut(df['Idade'], bins=caixas, labels=rotulos, right=True, include_lowest=True)

In [ ]:
df.columns

In [ ]:
ordem = [
    'ID', 'Salario', 'Idade', 'Faixa-Etaria', 'Estado_Civil', 'Formação', 'Criancas-Casa',
    'Adolescentes-Casa', 'Recencia', 'Qtde-Vinhos', 'Qtde-Frutas', 'Qtde-Carnes', 'Qtde-Peixes',
    'Qtde-Doces', 'Qtde-Premium', 'Qtde-Compras', 'Qtde-Compras-Web','Qtde-Compras-Catalogo',
    'Qtde-Compras-Loja', 'Visitas-Site-Mes', 'Reclamacoes', 'Gasto-Cliente'
]

df = df[ordem]

In [ ]:
df.head()

In [ ]:
df.to_csv('../data/processed/iFood.csv')

In [ ]:
df.to_parquet('../data/processed/iFood.parquet')

---

Análise Exploratória

In [2]:
df = pd.read_parquet('../data/processed/iFood.parquet')

In [ ]:
df.columns

In [16]:
# Hipótese 1. Clientes abaixo dos 30 anos gastam mais com produtos do iFood do que as outras
# faixas etárias.

gasto_etario = pd.pivot_table(
    df,
    index='Faixa-Etaria',
    values='Gasto-Cliente',
    aggfunc='sum'
).reset_index()

gasto_etario['Percentual'] = (gasto_etario['Gasto-Cliente'] / gasto_etario['Gasto-Cliente'].sum()) * 100

gasto_etario['Percentual'] = gasto_etario['Percentual'].apply(lambda x: f"{x:.2f}%")

fig = px.bar(
    gasto_etario,
    x='Faixa-Etaria',
    y='Gasto-Cliente',
    text=gasto_etario['Gasto-Cliente'].apply(lambda x: f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")),
    color_discrete_sequence=['royalblue']
)

fig.update_layout(
    title={
        'text': 'Clientes abaixo dos 30 anos gastam menos com produtos do iFood do que as outras faixas etárias.',
        'x': 0.5
    },
    xaxis_title='Faixa-Etaria',
    yaxis_title='SUM de Gasto-Cliente',
    yaxis_tickprefix='R$ ',
    yaxis_tickformat=',.0f',
    plot_bgcolor='white',
    showlegend=False,
    yaxis=dict(
        showgrid=True,
        gridcolor='black',
        gridwidth=1,
        range=[0, gasto_etario['Gasto-Cliente'].max() * 1.2],
        zeroline=True,             # <-- mostra a linha do eixo 0
        zerolinecolor='black',     # <-- define a cor
        zerolinewidth=1.5,
    )
)

fig.show()

C:\Users\pedro\AppData\Local\Temp\ipykernel_3504\2796518283.py:4: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



In [15]:
# Hipótese 2. Clientes solteiros gastam menos do que os outros segmentos de clientes.

gasto_civil = pd.pivot_table(
    df,
    index='Estado_Civil',
    values='Gasto-Cliente',
    aggfunc='sum'
).reset_index()

gasto_civil['Percentual'] = (gasto_civil['Gasto-Cliente'] / gasto_civil['Gasto-Cliente'].sum()) * 100

gasto_civil['Percentual'] = gasto_civil['Percentual'].apply(lambda x: f"{x:.2f}%")

fig = px.bar(
    gasto_civil,
    x='Estado_Civil',
    y='Gasto-Cliente',
    text=gasto_civil['Gasto-Cliente'].apply(lambda x: f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")),
    color_discrete_sequence=['royalblue'],
    category_orders={'Estado_Civil': ['Solteiro', 'Namorando', 'Casado', 'Divorciado', 'Viúvo(a)', 'Total']},
)

fig.update_layout(
    title={
        'text': 'Clientes solteiros gastam menos do que os outros segmentos de clientes.',
        'x': 0.5
    },
    xaxis_title='Estado Civil',
    yaxis_title='SUM de Gasto-Cliente',
    yaxis_tickprefix='R$ ',
    yaxis_tickformat=',.0f',
    plot_bgcolor='white',
    showlegend=False,
    yaxis=dict(
        showgrid=True,
        gridcolor='black',
        gridwidth=1,
        range=[0, gasto_civil['Gasto-Cliente'].max() * 1.2],
        tick0=0,
        dtick= gasto_civil['Gasto-Cliente'].max() / 8,
        zeroline=True,             # <-- mostra a linha do eixo 0
        zerolinecolor='black',     # <-- define a cor
        zerolinewidth=1.5,
    )
    # yaxis=dict()
)

fig.show()

In [19]:
# Hipótese 3. Clientes solteiros abaixo dos 30 anos gastam mais com produtos do iFood
# do que as outras faixas etárias.

gasto_etario_civil = pd.pivot_table(
    df,
    index='Faixa-Etaria',
    columns='Estado_Civil',
    values='Gasto-Cliente',
    aggfunc='sum'
)

gasto_etario_civil['Total'] = gasto_etario_civil.sum(axis=1)

ordem = ['Solteiro', 'Namorando', 'Casado', 'Divorciado', 'Viúvo(a)', 'Total']

gasto_etario_civil = gasto_etario_civil[ordem]

gasto_etario_civil_reset = gasto_etario_civil.reset_index().melt(
    id_vars='Faixa-Etaria',
    var_name='Estado_Civil',
    value_name='Gasto-Cliente'
)

# Remover Total
gasto_etario_civil_reset = gasto_etario_civil_reset[gasto_etario_civil_reset['Estado_Civil'] != 'Total']

# Criar o gráfico
fig = px.bar(
    gasto_etario_civil_reset,
    x='Faixa-Etaria',
    y='Gasto-Cliente',
    color='Estado_Civil',
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_layout(
    title={
        'text': 'Clientes solteiros abaixo dos 30 anos gastam mais com produtos do iFood do que as outras faixas etárias.',
        'x': 0.5
    },
    xaxis_title='Faixa Etária',
    yaxis_title='SUM de Gasto-Cliente',
    yaxis_tickprefix='R$ ',
    yaxis_tickformat=',.0f',
    plot_bgcolor='white',
    bargap=0.15,
    yaxis=dict(
        showgrid=True,
        gridcolor='black',
        gridwidth=1,
        range=[0, gasto_etario_civil_reset['Gasto-Cliente'].max() * 1.2],
        tick0=0,
        dtick= gasto_etario_civil_reset['Gasto-Cliente'].max() / 8,
        zeroline=True,             # <-- mostra a linha do eixo 0
        zerolinecolor='black',     # <-- define a cor
        zerolinewidth=1.5,
    )
)

fig.show()

C:\Users\pedro\AppData\Local\Temp\ipykernel_3504\1714930080.py:4: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



In [ ]:
# Hipótese 4. Clientes com crianças em casa compram mais pelo ifood.

gasto_criancas = pd.pivot_table(
    df,
    index='Criancas-Casa',
    values='Gasto-Cliente',
    aggfunc='sum'
)

# gasto_criancas
gasto_criancas_reset = gasto_criancas.reset_index()

fig = px.pie(
    gasto_criancas_reset,
    names='Criancas-Casa',
    values='Gasto-Cliente',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_traces(
    # textposition='inside',
    # textinfo='percent+label',
    hovertemplate='Crianças em Casa: %{label}<br>Gasto: R$ %{value:,.0f}<extra></extra>'
)

fig.add_annotation(
    text='Clientes sem filhos em casa representam 86% de todo o faturamento da campanha',
    xref="paper", yref="paper",
    x=0.5, y=-0.15,
    showarrow=False,
    font=dict(size=12),
    align="center"
)

fig.update_layout(
    legend_title_text='Qtde de Crianças em Casa',
    title={
        'text': 'Clientes com crianças em casa compram mais pelo iFood.',
        'x': 0.5
    }
)

fig.show()

In [ ]:
# Hipótese 5. Clientes que compram mais carne também compram mais vinho.

gasto_carne_vinho = df.groupby('ID')[['Qtde-Carnes','Qtde-Vinhos']].sum()

fig = px.scatter(
    gasto_carne_vinho.reset_index(),
    x='Qtde-Carnes',
    y='Qtde-Vinhos',
    title='Clientes que compram mais carne também compram mais vinho.',
    labels={
        'Qtde-Carnes': 'Quantidade de Carne',
        'Qtde-Vinhos': 'Quantidade de Vinho'
    },
    trendline='ols'
)

fig.data[1].update(
    line=dict(color='red', width=3)
)

fig.show()

In [ ]:
gasto_formacao = pd.pivot_table(
    df,
    index='Formação',
    values='Gasto-Cliente',
    aggfunc='sum'
).reset_index()

fig = plt.bar(
    
)

,Formação,Gasto-Cliente
0,Doutorado,304862
1,Fundamental,3184
2,Graduação,635455
3,Medio,89550
4,Mestrado,207845


In [18]:
# Faturamento por Formação.

gasto_formacao = pd.pivot_table(
    df,
    index='Formação',
    values='Gasto-Cliente',
    aggfunc='sum'
).reset_index()

fig = px.bar(
    gasto_formacao,
    x='Formação',
    y='Gasto-Cliente',
    text=gasto_formacao['Gasto-Cliente'].apply(lambda x: f"R$ {x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")),
    color_discrete_sequence=['royalblue'],
    category_orders={'Formação': ['Fundamental', 'Medio', 'Graduação', 'Mestrado', 'Doutorado']},
)

fig.update_layout(
    title={
        'text': 'Faturamento por Formação',
        'x': 0.5
    },
    xaxis_title='Formação',
    yaxis_title='SUM de Gasto-Cliente',
    yaxis_tickprefix='R$ ',
    yaxis_tickformat=',.0f',
    plot_bgcolor='white',
    showlegend=False,
    yaxis=dict(
        showgrid=True,
        gridcolor='black',
        gridwidth=1,
        range=[0, gasto_formacao['Gasto-Cliente'].max() * 1.2],
        zeroline=True,             # <-- mostra a linha do eixo 0
        zerolinecolor='black',     # <-- define a cor
        zerolinewidth=1.5,
    )
)

fig.show()

---

---